# Linear regression and Correlation Tutorial

In [0]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pandas as pd

## Correlation

Before we unpack the important concept of **correlation**, let's first refresh our memories about the concept of **independence**. We can understand this colloquially as the lack of any  relationship between two events or, for our purposes, two random variables: the realisation of one has no impact on the realisation of another. In mathematical and probabilistic terms, if two random variables $X$ and $Y$ are independent, then:

$$P(X=x|Y=y) = P(X=x).$$

This has the handy implication, as pointed out above, that $E(XY) = E(X) E(Y)$.

On the other hand, what if we defined $Y = 2X$? $Y$ is still a random variable whose realisation is uncertain, but it will bear an obvious relation to $X$. $E(2X^2) \ne E(X) E(Y)$. The higher that $X$ is, the higher that $Y$ will be, and in fact they will move in perfect lockstep: we refer to these random variables as being **perfectly positively correlated** (don't worry, we'll unpack that in more detail in a moment). And if $Y = -2X$, then $X$ and $Y$ would be **perfectly negatively correlated**: an increase in $X$ gives rise to a decrease in $Y$.

Before we get onto correlation, let's first get acquainted with the related concept of **covariance**. You already know that variance is a measure of the variability, or 'spread', of a statistical distribution; covariance describes the joint variability of two random variables. The covariance between $X$ and $Y$, with the $\mu$ terms defined as usual, is defined as:

\begin{align}
\text{Cov}_{X,Y} &= E[X - \mu_X].E[(Y - \mu_Y] \\
                 &= E(XY) - \mu_X \mu_Y
\end{align}

The sign of the covariance tells us something about the linear dependence between variables: if it's positive then they tend to move in the same direction, while a negative value indicates a tendency to move in opposite directions. But as the size of the covariance is influenced by the size of the observations, the strength of this linear dependence is best understood by a measure which standardises for this, and this is where correlation comes in.

There are in fact several statistical measures of correlation, but for our purposes at this point we focus only on the most common and well-known (and what most people mean when they talk about correlation), the **Pearson correlation coefficient** which is defined as follows for $X$ with mean $\mu_X$ and variance $\sigma^2_X$ and $Y$ with mean $\mu_Y$ and variance $\sigma^2_Y$:

$$\rho_{X,Y} = \frac{\text{Cov}_{X,Y}}{\sigma_X \sigma_Y}.$$

$\rho_{X,Y}$ takes on a value in the range [-1, 1], where -1 indicates perfect negative correlation, 1 perfect positive correlation and 0 a complete absence of correlation. Independent random variables will have a correlation coefficient of 0, as you'd expect, but a zero correlation does not necessarily imply independence: this is getting a little beyond the scope of this section of the course, but the key here is that the correlation coefficient measures linear dependence, but there may be other forms of dependence which do not reflect in the linear measure.

We may also compute a **sample correlation coefficient**, usually written as $r$, as follows for two samples of $X$ and $Y$ with $n$ observations of each and sample variances $s_x^2$ and $s_y^2$ calculated in the usual way:

$$r_{xy} = \displaystyle \frac{\sum_{i=1}^n (x_i - \bar{x}) (y_i - \bar{y})}{(n-1) s_x s_y} = \frac{\sum_{i=1}^n (x_i - \bar{x}) (y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2 (y_i - \bar{y})^2}}.$$



### Correlation vs. causation

The simple fact that two variables are strongly correlated (positively or negatively) does not necessarily imply that one causes another:

* Even where there is a causal relationship, there may be times when it is difficult to be sure which way the causation runs.

* There may be an association that is driven via another variable: one lovely data set (available at https://rdrr.io/github/rpruim/Lock5Data/man/MinistersRum.html) shows the very close relationship between the number of Methodist ministers living in New England over the 80 years from 1860 and the number of barrels of rum imported into Boston over the same time period. Clearly both are by-products of general population growth, rather than there being any causal relationship between them.

* The correlation may simply be spurious, the sort of weird relationship that is going to be observed at random from time to time even though there is no real relationship of any sort between the variables. For some fun illustrations of these, visit http://www.tylervigen.com/spurious-correlations. We reproduce one very cool relationship from that website below:


<img src="https://github.com/Explore-AI/Pictures/blob/master/chart.png?raw=true" alt="Drawing" style="width: 1000px;"/>


## Linear regression

Suppose that we've discovered some correlation between two variables, say the amount of work that Explore Data Science Academy students put into understanding linear regression and their overall performance on that section, and we'd now like to investigate whether we can formalise the relationship between them in such a way that it allows us to predict what the likely academic outcome will be for a student who exerts a given amount of effort on her studies. Formally, we are going to regress the values of the dependent or target variable, academic performance in this case, on the predictor or explanatory variable, hours of study effort (this is sometimes, a little confusingly, called the independent variable). The simplest form of this approach is known as **linear regression**, which assumes that there is a linear relationship between the target variable $y$ and the explanatory variable $x$ which may be represented by:

$$ y = \alpha + \beta x.$$

$\alpha$ is clearly the y-intercept, and represents the part of the outcome which is independent of variability in the explanatory variable. $\beta$ measures the strength of the relationship between the two variables.

Typically there is no straight line $\alpha + \beta x$ which will pass through all of the observed $y$ values, so the challenge is to estimate the $\alpha$ and $\beta$ which give the best fit to the data. The problem of course is that we have no unequivocal guide as to what constitutes "best" in this regard. Clearly we want a line that passes as close as possible to the data in some way. This is typically achieved for linear regression by summing the squares of the differences between the observed and predicted values, and minimising this sum of squared errors.

### Least squares optimisation

We call this method of optimisation (i.e. maximising how good our model is) the **least squares** method. Why least squares, and not least absolute differences, or least cubes? There are a number of reasons, which are beyond our scope, so let's just accept for now that this is the approach that we must take when carrying out linear regression.

Let's start by having a look at a data set and then setting about fitting a linear regression to it. What we'll do is simulate a data set from a known distribution, and then fit a linear model to it. We'll assume that the data are generated by a process as follows:

$$y_i = 10 + 2 x_i + \epsilon_i,$$

where $\epsilon_i \sim N(0,100)$ and the $\epsilon_i$ are what we call **i.i.d** (independent and identically distributed). This is just a random noise term; without it, every observation of $y$ would like on the straight line $10 + 2x$, but this term introduces some variation arond this line, similar to what we'd expect with real-world data. Since we know that the underlying linear model is $y = 10 + 2x$, we should expect that our linear regression should come to a solution similar to this. Let's see how we go:

In [0]:
np.random.seed(100)                                             # set seed for reproducibility
x = np.arange(1,51)                                             # x from 1 to 50
e = np.random.normal(loc = 0, scale = np.sqrt(100), size = 50)  # simulate the i.i.d epsilon ~ N(0,10)
y = 10 + 2 * x + e                                              # calculate y

plt.rcParams["figure.figsize"] = (10,6)
plt.plot(x, y, 'ko', label = "Observed")
plt.plot(x, 10 + 2 * x, 'r', label = 'Underlying model y = 10 + 2x')
plt.legend()
plt.show()

If we fit a model of the form $y = \alpha + \beta x$, then we are looking to minimise the sum of the error terms

$$e = \sum_{i=1}^n \left( y_i - (\alpha + \beta x_i) \right)^2.$$

We typically write predicted values from the model as $\hat{y}$ so this is equivalent to $\sum_{i=1}^n \left( y_i - \hat{y_i} \right)^2.$

So how do we minimise this sum squared error term? By setting the partial derivatives of this equation, with respect ot $\alpha$ and $\beta$ respectively, to zero, and solving the result set of simultaneous equations:

\begin{align}
\frac{\partial e}{\partial \alpha} &= -2 \sum_{i=1}^n (y_i - \alpha - \beta x_i) = 0 \\
\frac{\partial e}{\partial \beta}  &= -2 \sum_{i=1}^n x_i (y_i - \alpha - \beta x_i) = 0 
\end{align}

From these we have that:
\begin{align}
\sum_{i=1}^n y_i &= n \alpha + \beta \sum_{i=1}^n x_i \\
\sum_{i=1}^n x_i y_i &= \alpha \sum_{i=1}^n x_i + \beta \sum_{i=1}^n x_i^2
\end{align}

We can then solve for:

\begin{align}
\hat{\beta} &= \displaystyle \frac{\sum_{i=1}^n x_i y_i - \frac{\sum_{i=1}^n x_i \sum_{i=1}^n y_i}{n}}{\sum_{i=1}^n x_i^2 - \frac{(\sum_{i=1}^n x_i)^2}{n}}  \\
\hat{\alpha} &= \bar{y} - \beta \bar{x}
\end{align}

Let's try this out with our data set.


In [0]:
sumx = sum(x)
sumy = sum(y)
sumxy = sum(x * y)
sumx2 = sum(x**2)
n = len(x)


beta = (sumxy - sumx * sumy / n) / (sumx2 - sumx**2 / n)
alpha = np.mean(y) - beta * np.mean(x)
print(alpha)
print(beta)

So the best fitting model to the data is $y = 12.4 + 1.89 x$. Our finite sample, with its random variation, results in an estimated model which differs from the linear model we know was used to generate the data (this is of course something we don't know in the real world).

We can also do this directly using the appropriate module from ```Scikit Learn```.

In [0]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
x = x.reshape((50,1))      # reshaping required for inputs to LinearRegression.fit()
np.shape(x)
model = lm.fit(x,y)
print('Coefficients: \n', model.coef_)
y_pred = model.predict(x)
# why can't I get the intercept from model.coef_?

plt.rcParams["figure.figsize"] = (10,6)
plt.plot(x, y, 'ko', label = "Observed")
plt.plot(x, 10 + 2 * x, 'r', label = "Underlying model: y = 10 + 2x")
plt.plot(x, y_pred, '--r', label = "Linear regression: y = 12.4 + 1.89x ")
plt.legend()
plt.show()

As a side note: we have here used our entire data set to estimate the linear model. As you will learn, in practice what we usually do is to split our data, and only use part of it for **training** the model (what we've done above), leaving the rest available for **testing**. We'll get onto the important concepts of bias and variance, and the reasons for following this approach, a little later on in the course.

### TSS, RSS and ESS

Let's define a few terms now:

The **total sum of squares (TSS)** is defined as the total variability of y:

$$\text{TSS } = \sum_{i=1}^n (y_i - \bar{y})^2.$$

We then define the **error sum of squares (ESS)** as the sum of squares of the differences between our $y$ values and those estimated from our fitted model (this is of course the value we aim to minimise through linear regression):

$$\text{ESS } = \sum_{i=1}^n (y_i - \hat{y_i}) ^2$$

wher $\hat{y}$ represents the predicted $y$ values from our linear model.

The difference between these two, then, is the amount of variability in $y$ that is explained by our linear model, and we call this the **regression sum of squares (RSS)**:

$$\text{RSS = TSS - ESS = }\sum_{i=1}^n (\hat{y_i} - \bar{y}) ^2.$$


### $R^2$ as a measure of fit

The final new concept for this tutorial is **$R^2$, the coefficient of determination**. This is the proportion of the total variance in $y$ explained by the linear model, in other words:

\begin{align}
R^2 &= \frac{\text{RSS}}{\text{TSS}} \\
    & = \frac{\sum_{i=1}^n (\hat{y_i} - \bar{y}) ^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
\end{align}

The higher the proportion of variance we can explain with our model, the better a job it is doing of explaining the data, so as high an $R^2$ as possible is desirable. Let's calculate it for our example above:

In [0]:
tss = sum((y - np.mean(y))**2)
rss = sum((y_pred - np.mean(y))**2)

print(rss/tss)

There is also a ```Scikit Learn``` function we can invoke to estimate that directly.

In [0]:
from sklearn.metrics import r2_score
r2_score(y, y_pred)

Wondering why the name $R^2$? Well, remember that we labelled the sample correlation coefficient $r$? Lok what happens when we calculate the correlation between $y$ and $\hat{y}$, and then square it:

In [0]:
r = np.corrcoef(y, y_pred)[0,1]
print(r**2)

### Non-linearity

One inherent weakness of linear regression, which you might already have recognised, is right there in its name: we are fitting a linear model to data which might show signs of a non-linear relationship. We'll be learning about more flexible algorithms later on in your course, but it's worth noting for now that some of those non-linearities can be catered for by a linear regression, simply by making a sutiable transformation of the data. For example, if we take the data appear to exhibit an exponentially increasing relationship, we might find that regressing $\log y$ (rather than $y$ itself) on $x$ gives us a reasonable model. 

### Multiple regression



Of course, there are real-world situations where it would be reasonable to assume that more than one explanatory variable affects the target variable. This leads us into a discussion of multiple regression, which we're going to leave over until the RSA Stock Market sprint in a few weeks' time.

What that means is that you've made it to the end of the hypothesis testing & linear regression tutorials. Well done! Go and take a well-earned break, and come back to make sure you've bedded down all of this material.

But first, coffee!

![coffee](https://github.com/Explore-AI/Pictures/blob/master/coffeebreak.png?raw=true)

